# Generating all sound files needed for musicpy using one file.
Note: this is extremely fragile and only works if you restart kernel -> run all -> restart kernel

In [ ]:
from musicpy.daw import *
import musicpy.daw as daw_module
from io import BytesIO
from pydub import AudioSegment

# Monkey-patch to fix librosa API compatibility issue
# In librosa 0.10+, pitch_shift requires keyword-only arguments (sr= and n_steps=)
original_pitch_shift = daw_module.pitch.pitch_shift

def patched_pitch_shift(self, semitones, mode='librosa'):
    """Patched version that uses keyword arguments for librosa"""
    if mode == 'librosa':
        import librosa
        import soundfile
        
        # Use keyword arguments for librosa 0.10+ compatibility
        data_shifted = librosa.effects.pitch_shift(
            self.audio,
            sr=self.sample_rate,
            n_steps=semitones
        )
        
        # Write to BytesIO buffer
        current_sound = BytesIO()
        soundfile.write(current_sound, data_shifted, self.sample_rate, format='WAV')
        current_sound.seek(0)
        
        # Convert BytesIO to AudioSegment (what musicpy expects)
        audio_segment = AudioSegment.from_wav(current_sound)
        return audio_segment
    else:
        # Fall back to original for other modes
        return original_pitch_shift(self, semitones, mode)

# Apply the patch
daw_module.pitch.pitch_shift = patched_pitch_shift


In [ ]:
sound_path = 'sounds/testtone.wav'
pitch = pitch(sound_path, note='C5', format=None)

In [ ]:
pitch.export_sound_files(path='.', folder_name="custom_tone", start='A0', end='C8', format='wav', mode='librosa')